In [1]:
import io
import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 20
pd.options.display.max_rows = 4

In [2]:
%%time
file_path = './data/processed/dataset.pkl.zst'
DF_FULL = pd.read_pickle(file_path)
DF_FULL

CPU times: user 100 ms, sys: 37.1 ms, total: 137 ms
Wall time: 137 ms


,CD_CVM,CNPJ_CIA,DENOM_CIA,fs_type,is_cons,DT_REFER,VERSAO,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,94,92.693.019/0001-89,PANATLANTICA S.A.,1,True,2010-12-31,2,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,211293000.0
1,94,92.693.019/0001-89,PANATLANTICA S.A.,1,False,2010-12-31,2,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,191541000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6783058,518700,70.216.429/0001-04,BRAMEX - BRASIL ...,7,False,2013-12-31,1,2013-01-01,2013-12-31,0,7.08.04.03,Lucros Retidos /...,True,NaN,-1200611.0
6783059,518700,70.216.429/0001-04,BRAMEX - BRASIL ...,7,False,2013-12-31,1,2013-01-01,2013-12-31,0,7.08.05,Outros,True,NaN,486752.0


In [3]:
DF_FULL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6783060 entries, 0 to 6783059
Data columns (total 15 columns):
 #   Column         Dtype   
---  ------         -----   
 0   CD_CVM         category
 1   CNPJ_CIA       category
 2   DENOM_CIA      category
 3   fs_type        category
 4   is_cons        category
 5   DT_REFER       category
 6   VERSAO         category
 7   DT_INI_EXERC   category
 8   DT_FIM_EXERC   category
 9   ORDEM_EXERC    category
 10  CD_CONTA       category
 11  DS_CONTA       category
 12  ST_CONTA_FIXA  category
 13  COLUNA_DF      category
 14  VL_CONTA       category
dtypes: category(15)
memory usage: 210.8 MB


In [4]:
# df_dataset.query("CD_CVM == 9512")
# df_dataset.query("VL_CONTA == 0")
DF_FULL.VL_CONTA.nunique()


987144

In [26]:
class Company():
    def __init__(self, cvm_number: int):
        self.cvm_number = int(cvm_number)
        self._df = DF_FULL.query("CD_CVM == @self.cvm_number").copy()
        self.remove_category()
    def remove_category(self):
        self._df = self._df.astype({
            'CD_CVM': 'object',
            'fs_type': np.int8,
            'is_cons': bool,
            'DT_REFER': 'datetime64',
            'DT_INI_EXERC': 'datetime64',
            'DT_FIM_EXERC': 'datetime64',
            'ORDEM_EXERC': np.int8,
            'CD_CONTA': 'object',
            'DS_CONTA': 'object',
            'ST_CONTA_FIXA': bool,
            'COLUNA_DF': 'object',
            'VL_CONTA': float
        })
        return

    def assets(self) -> pd.DataFrame:
        df = self._df.query("fs_type == 1").copy()
        df.drop(columns=['CNPJ_CIA', 'DENOM_CIA', 'VERSAO'], inplace=True)
        # df = self.remove_category()
        # df = df_dataset.query("CD_CVM == @self.cvm_number")
        df.sort_values(['DT_REFER', 'ORDEM_EXERC'], inplace=True)
        df.reset_index(drop=True, inplace=True)
        return df


In [27]:
%%time
petro = Company(9512)
df = petro.assets()
df

CPU times: user 25.7 ms, sys: 5.14 ms, total: 30.8 ms
Wall time: 30.1 ms


,CD_CVM,fs_type,is_cons,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,9512,1,True,2010-12-31,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,3.504189e+11
1,9512,1,False,2010-12-31,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,3.189971e+11
...,...,...,...,...,...,...,...,...,...,...,...,...
2448,9512,1,True,2021-09-30,NaT,2021-09-30,0,1.02.04,Intangível,True,NaN,1.660100e+10
2449,9512,1,False,2021-09-30,NaT,2021-09-30,0,1.02.04,Intangível,True,NaN,1.644500e+10


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2450 entries, 0 to 2449
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   CD_CVM         2450 non-null   object        
 1   fs_type        2450 non-null   int8          
 2   is_cons        2450 non-null   bool          
 3   DT_REFER       2450 non-null   datetime64[ns]
 4   DT_INI_EXERC   0 non-null      datetime64[ns]
 5   DT_FIM_EXERC   2450 non-null   datetime64[ns]
 6   ORDEM_EXERC    2450 non-null   int8          
 7   CD_CONTA       2450 non-null   object        
 8   DS_CONTA       2450 non-null   object        
 9   ST_CONTA_FIXA  2450 non-null   bool          
 10  COLUNA_DF      0 non-null      object        
 11  VL_CONTA       2450 non-null   float64       
dtypes: bool(2), datetime64[ns](3), float64(1), int8(2), object(4)
memory usage: 162.8+ KB


In [11]:
df = df.astype({
    'CD_CVM': 'object',
    'fs_type': np.int8,
    'is_cons': bool,
    'DT_REFER': 'datetime64',
    'DT_INI_EXERC': 'datetime64',
    'DT_FIM_EXERC': 'datetime64',
    'ORDEM_EXERC': np.int8,
    'CD_CONTA': 'object',
    'DS_CONTA': 'object',
    'ST_CONTA_FIXA': bool,
    'COLUNA_DF': 'object',
    'VL_CONTA': float
})
df

,CD_CVM,fs_type,is_cons,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,9512,1,True,2010-12-31,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,3.504189e+11
1,9512,1,False,2010-12-31,NaT,2009-12-31,-1,1,Ativo Total,True,NaN,3.189971e+11
...,...,...,...,...,...,...,...,...,...,...,...,...
2448,9512,1,True,2021-09-30,NaT,2021-09-30,0,1.02.04,Intangível,True,NaN,1.660100e+10
2449,9512,1,False,2021-09-30,NaT,2021-09-30,0,1.02.04,Intangível,True,NaN,1.644500e+10


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2450 entries, 0 to 2449
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   CD_CVM         2450 non-null   object        
 1   fs_type        2450 non-null   int8          
 2   is_cons        2450 non-null   bool          
 3   DT_REFER       2450 non-null   datetime64[ns]
 4   DT_INI_EXERC   0 non-null      datetime64[ns]
 5   DT_FIM_EXERC   2450 non-null   datetime64[ns]
 6   ORDEM_EXERC    2450 non-null   int8          
 7   CD_CONTA       2450 non-null   object        
 8   DS_CONTA       2450 non-null   object        
 9   ST_CONTA_FIXA  2450 non-null   bool          
 10  COLUNA_DF      0 non-null      object        
 11  VL_CONTA       2450 non-null   float64       
dtypes: bool(2), datetime64[ns](3), float64(1), int8(2), object(4)
memory usage: 162.8+ KB


In [13]:
df.DT_REFER.min()

Timestamp('2010-12-31 00:00:00')

In [57]:
%%time
c_types = df.dtypes
c_types

CPU times: user 236 µs, sys: 0 ns, total: 236 µs
Wall time: 240 µs


CD_CVM       category
fs_type          int8
               ...   
COLUNA_DF    category
VL_CONTA      float64
Length: 12, dtype: object

In [63]:
for column, c_type in zip(df.columns, df.dtypes):
    if (c_type != 'int8') and (c_type != 'bool') and (c_type != 'float64'):
        print(column, c_type)
        df[f'{column}'] = df[f'{column}'].astype('category')

CD_CVM int32
DT_REFER datetime64[ns]
DT_INI_EXERC datetime64[ns]
DT_FIM_EXERC datetime64[ns]
CD_CONTA object
DS_CONTA object
COLUNA_DF object


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37114 entries, 0 to 37113
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   CD_CVM         37114 non-null  category
 1   fs_type        37114 non-null  int8    
 2   is_cons        37114 non-null  bool    
 3   DT_REFER       37114 non-null  category
 4   DT_INI_EXERC   25026 non-null  category
 5   DT_FIM_EXERC   37114 non-null  category
 6   ORDEM_EXERC    37114 non-null  int8    
 7   CD_CONTA       37114 non-null  category
 8   DS_CONTA       37114 non-null  category
 9   ST_CONTA_FIXA  37114 non-null  bool    
 10  COLUNA_DF      13916 non-null  category
 11  VL_CONTA       37114 non-null  float64 
dtypes: bool(2), category(7), float64(1), int8(2)
memory usage: 803.3 KB
